In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Define quantile loss function
def quantile_loss(y_true, y_pred, quantile=0.9):
    e = y_true - y_pred
    return tf.keras.backend.mean(tf.keras.backend.maximum(quantile * e, (quantile - 1) * e), axis=-1)

# Load data
df = pd.read_csv('panama_dataset.csv')
df.dropna(inplace=True)

# Preprocess data
s1 = MinMaxScaler(feature_range=(0,1))
Xs = s1.fit_transform(df)
s2 = MinMaxScaler(feature_range=(0,1))
ys = s2.fit_transform(df[['target_variable']])



# Split data into training and testing sets
X_train1, X_test1, y_train, y_test = train_test_split(Xs, ys, test_size=0.3, shuffle=False)
print('Train set shape', X_train1.shape)
print('testing set shape', X_test1.shape)



C:\Users\hp\AppData\Local\Temp\ipykernel_13196\2812130262.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



Train set shape (33633, 5)
testing set shape (14415, 5)


In [2]:
X_train=np.reshape(X_train1,(33633,5,1))
X_test=np.reshape(X_test1,(14415,5,1))

In [3]:
# Define combined hybrid model
def hybrid_model(input_shape, quantile=0.9):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Shared LSTM layers
    lstm_out = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)
    lstm_out = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)

    
    # Attention mechanism
    attention1 = tf.keras.layers.Attention()([lstm_out, lstm_out])
    attention1 = tf.keras.layers.GlobalAveragePooling1D()(attention1)
    
    # Output layer
    quantile_lstm_out = tf.keras.layers.Dense(1)(attention1)
    
    # Quantile LSTM model
    quantile_lstm_model = tf.keras.Model(inputs=inputs, outputs=quantile_lstm_out)
    quantile_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                                loss=lambda y_true, y_pred: quantile_loss(y_true, y_pred, quantile=quantile))
    
    return quantile_lstm_model

# Train the hybrid model
model = hybrid_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train, epochs=200, batch_size=32)

# Predictions
y_pred = model.predict(X_test)

y_pred = s2.inverse_transform(y_pred)
y_test = s2.inverse_transform(y_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100
mse = mean_squared_error(y_test, y_pred)
nrmse = (rmse/np.mean(y_pred))*100

print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE:", mape)
print("MSE:", mse)


Epoch 1/200

1052/1052 [==============================] - 13s 6ms/step - loss: 0.0896
Epoch 2/200
1052/1052 [==============================] - 6s 6ms/step - loss: 0.0099
Epoch 3/200
1052/1052 [==============================] - 6s 6ms/step - loss: 0.0094
Epoch 4/200
1052/1052 [==============================] - 6s 6ms/step - loss: 0.0085
Epoch 5/200
1052/1052 [==============================] - 6s 6ms/step - loss: 0.0069
Epoch 6/200
1052/1052 [==============================] - 6s 6ms/step - loss: 0.0053
Epoch 7/200
1052/1052 [==============================] - 7s 7ms/step - loss: 0.0045
Epoch 8/200
1052/1052 [==============================] - 7s 7ms/step - loss: 0.0041
Epoch 9/200
1052/1052 [==============================] - 6s 6ms/step - loss: 0.0037
Epoch 10/200
1052/1052 [==============================] - 6s 6ms/step - loss: 0.0035
Epoch 11/200
1052/1052 [==============================] - 7s 6ms/step - loss: 0.0033
Epoch 12/200
1052/1052 [==============================] - 8s 7ms/step -

In [4]:
y_pred= pd.DataFrame(y_pred)
y_test= pd.DataFrame(y_test)

In [5]:
result_df = pd.concat([y_pred, y_test], axis=1)
result_df.to_csv(r"C:\Users\hp\Desktop\New folder\PANAMA\DAQ-LSTM_output(p3).csv", index=False)